# UCI Adult Dataset or Census Income

This is a very popular ML task, with tabular data. The objective is to predict whether income exceeds $50K/yr based on census data. 
Also known as "Census Income" dataset.

The data is old and biased on different ways ... but it can be used opaquely for ML experimentation.



## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

GoMLX provides [a simple `adult` library](https://pkg.go.dev/github.com/gomlx/gomlx/examples/adult) to facilitate downdoaling and preprocessing the data. Data is available in [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Adult).

After downloading the data and validating the checksum (both training and testing), it generates the quantiles for the continuous features, and the vocabularies for the categorical features. It saves all this info for faster restart later in a binary file. So this won't be necessary a second time.

The quantiles are used to calibrate the values, using a piece-wise-lienar calibration, very good for these things. See [`layers.PieceWiseLinearCalibration` documentation](https://pkg.go.dev/github.com/gomlx/gomlx@v0.1.0/ml/layers#PieceWiseLinearCalibration).

We create a flag `--data` to define the directory where to save the intermediary files: downloaded and preprocessed datasets.
In this examle we set it to `~/work/uci-adult`. Verbosity can be contolled with the `--verbosity` flag. 

We set default in Go for these flags, but they can easily be reset for a new run by providing them after the `%%` Jupyter kernel meta-command -- in indicates that the subsequent lines should be put in to a `func main`.


In [2]:
import (
    "flag"
    
    "github.com/gomlx/gomlx/examples/adult"
)

var (
    flagDataDir       = flag.String("data", "~/work/uci-adult", "Directory to save and load downloaded and generated dataset files.")
    flagVerbosity     = flag.Int("verbosity", 0, "Level of verbosity, the higher the more verbose.")
    flagForceDownload = flag.Bool("force_download", false, "Force re-download of Adult dataset files.")
    flagNumQuantiles  = flag.Int("quantiles", 100, "Max number of quantiles to use for numeric features, used during piece-wise linear calibration. It will only use unique values, so if there are fewer variability, fewer quantiles are used.")
)

%% --verbosity=2
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)


Sample Categorical: (24.08% positive ratio, 23.86% weighted positive ratio)
	Row 0:	[7 10 5 1 2 5 2 39]
	Row 1:	[6 10 3 4 1 5 2 39]
	Row 2:	[4 12 1 6 2 5 2 39]
	...
	Row 32558:	[4 12 7 1 5 5 1 39]
	Row 32559:	[4 12 5 1 4 5 2 39]
	Row 32560:	[5 12 3 4 6 5 1 39]

Sample Continuous:
	Row 0:	[39 13 2174 0 40]
	Row 1:	[50 13 0 0 13]
	Row 2:	[38 9 0 0 40]
	...
	Row 32558:	[58 9 0 0 40]
	Row 32559:	[22 9 0 0 20]
	Row 32560:	[52 9 15024 0 40]


In [3]:
!ls -lh ~/work/uci-adult

total 7.0M
-rw-r--r-- 1 jupyter  1000 3.8M Mar 21  2023 adult.data
-rw-r--r-- 1 jupyter  1000 2.0M Mar 21  2023 adult.test
-rw-r--r-- 1 jupyter  1000 1.3M Mar 21  2023 adult_data-100_quantiles.bin
drwxr-x--- 2 jupyter users 4.0K Nov 26 11:40 base_model


### Creating Datasets

First we create the GoMLX's `Manager`: it's the object that manages the underlying XLA
setup, connection and execution. It's needed to create tensors.

With that we create the samplers of data that we will use to train and evaluate. They implement 
GoMLX's `train.Dataset` interface, which is what is used by our training loop to draw batches to
train, or our eval loop to draw batches to evaluate.

The inputs are 3 tensors: *categorical values*, *continuous values* and *weights*.

In the cell below we define the `Manager` flags, `BuildSamplers` and printout some samples.

In [4]:
import (
    "flag"
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/tensor"
)

var (
    flagBatchSize      = flag.Int("batch", 128, "BatchSampler size for training")
)

// Global manager created an initialization, used everywhere.
var manager = NewManager()

// BuildDatasets returns 3 `train.Dataset`:
// * trainingSampler is an endless random sampler used for training.
// * trainingEvalSampler samples through exactly one epoch of the train dataset.
// * testEvalSampler samples through exactly one epoch of the test dataset.
func BuildDatasets(manager *Manager) (trainDS, trainEvalDS, testEvalDS train.Dataset) {
    baseDS := adult.NewDataset(manager, adult.Data.Train, "batched train")
    trainEvalDS = baseDS.Copy().BatchSize(*flagBatchSize, false)
    testEvalDS = adult.NewDataset(manager, adult.Data.Test, "test").
        BatchSize(*flagBatchSize, false)
    // For training, we shuffle and loop indefinitely.
    trainDS = baseDS.BatchSize(*flagBatchSize, true).Shuffle().Infinite(true)
    return
}

// PositiveRatio finds out the the ratio of positive labels in the
// training and testing data.
//
// We could do this easily with GoMLX computation model (just `ReduceAllSum`), but
// this examples shows it's also ok to mix Go computations.
func PositiveRatio(ds train.Dataset) float32 {
    ds.Reset()  // Start from beginning.
    var sum float32
    var count float32
    for {
        _, _, labels, err := ds.Yield()
        if err == io.EOF {
            break;
        }
        if err != nil { panic(err) }
        data := labels[0].Local().FlatCopy().([]float32)
        for _, value := range data {
            sum += value
        }
        count += float32(len(data))
    }
    return sum/count
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
trainingDS, trainingEvalDS, testEvalDS := BuildDatasets(manager)

// Take one batch.
_, inputs, labels, err := trainingDS.Yield()
if err != nil { panic(err) }
fmt.Printf("Inputs of batch (size %d):\n", *flagBatchSize)
fmt.Printf("\tcategorical:\n\t\tFeatures=%v\n", adult.Data.VocabulariesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[0].Local().StringN(16))
fmt.Printf("\tcontinuous:\n\t\tFeatures=%v\n", adult.Data.QuantilesFeatures)
fmt.Printf("\t\tValues: %s\n", inputs[1].Local().StringN(10))
fmt.Printf("\tweights: %s\n", inputs[2].Local().StringN(5))
fmt.Printf("\nLabels of batch:\n\t%s\n", labels[0].Local().StringN(10))
fmt.Printf("\nLabels distributions:\n\tTrain:\t%.2f%% positive\n\tTest:\t%.2f%% positive\n",
           PositiveRatio(trainingEvalDS)*100.0, PositiveRatio(testEvalDS)*100.0)


Inputs of batch (size 128):
	categorical:
		Features=[workclass education marital-status occupation relationship race sex native-country]
		Values: (Int64)[128 8]: (... too large, 1024 values ..., first 16 values: [6 13 3 5 1 5 2 39 4 7 3 5 1 3 2 39])
	continuous:
		Features=[age education-num capital-gain capital-loss hours-per-week]
		Values: (Float32)[128 5]: (... too large, 640 values ..., first 10 values: [50 14 2407 0 98 63 5 0 0 40])
	weights: (Float32)[128 1]: (... too large, 128 values ..., first 5 values: [30653 201631 56915 65389 340940])

Labels of batch:
	(Float32)[128 1]: (... too large, 128 values ..., first 10 values: [0 0 0 0 1 1 0 0 1 0])

Labels distributions:
	Train:	24.08% positive
	Test:	23.62% positive


## Model Definition

Lots of hyper-parameter flags, but otherwise a straight forward FNN, using piece-wise linear calibration of the continuous features, and embeddings for the categorical features.

> **Note**: building models is a constant checking that shapes are compatible. It's a bit annoying, in particular because shapes are known in runtime only -- no compile time check. GoMLX tries to help providing a stack trace of where errors happen so one can pin-point issues quickly. But often it involves lots of experimentation (more than ordinary Go code).
>
> Developing with a Noteboook (see [GoNB](https://github.com/janpfeifer/gonb)) or simply a unit test on your `ModelGraph` function are quick/convenient ways to develop models -- before actually training them. You can also use shape asserts in the middle of the 
>`ModelGraph`, as we do below.

In [5]:
import (
    "fmt"
    "io"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/ml/train/optimizers"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    // ModelDType used for the model. Must match RawData Go types.
    ModelDType = shapes.Float32
    

    // Model hyperparameters.
    flagUseCategorical       = flag.Bool("use_categorical", true, "Use categorical features.")
    flagUseContinuous        = flag.Bool("use_continuous", true, "Use continuous features.")
    flagTrainableCalibration = flag.Bool("trainable_calibration", true, "Allow piece-wise linear calibration to adjust outputs.")
    flagEmbeddingDim    = flag.Int("embedding_dim", 8, "Default embedding dimension for categorical values.")
    flagNumHiddenLayers = flag.Int("hidden_layers", 8, "Number of hidden layers, stacked with residual connection.")
    flagNumNodes        = flag.Int("num_nodes", 32, "Number of nodes in hidden layers.")
    flagDropoutRate     = flag.Float64("dropout", 0, "Dropout rate")
    
    // Training parameter, referenced here.
    flagLearningRate  = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagNumSteps      = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
)


// ModelGraph outputs the logits (not the probabilities). The parameter inputs should contain 3 tensors:
//
// - categorical inputs, shaped  `(int64)[batch_size, len(VocabulariesFeatures)]`
// - continuous inputs, shaped `(float32)[batch_size, len(Quantiles)]`
// - weights: not currently used, but shaped `(float32)[batch_size, 1]`.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used, since the dataset is always the same.
    g := inputs[0].Graph()
    
    // Use Cosine schedule of the learning rate.
    optimizers.CosineAnnealingSchedule(ctx, g, ModelDType).
        PeriodInSteps(*flagNumSteps/3).Done()
    
    categorical, continuous := inputs[0], inputs[1]
    batchSize := categorical.Shape().Dimensions[0]
    
    var allEmbeddings []*Node

    if *flagUseCategorical {
        // Embedding of categorical values, each with its own vocabulary.
        numCategorical := categorical.Shape().Dimensions[1]
        for catIdx := 0; catIdx < numCategorical; catIdx++ {
            // Take one column at a time of the categorical values.
            split := Slice(categorical, AxisRange(), AxisRange(catIdx, catIdx+1))
            // Embed it accordingly.
            embedCtx := ctx.In(fmt.Sprintf("categorical_%d_%s", catIdx, adult.Data.VocabulariesFeatures[catIdx]))
            vocab := adult.Data.Vocabularies[catIdx]
            vocabSize := len(vocab)
            embedding := layers.Embedding(embedCtx, split, ModelDType, vocabSize, *flagEmbeddingDim)
            embedding.AssertDims(batchSize, *flagEmbeddingDim) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, embedding)
        }
    }

    if *flagUseContinuous {
        // Piecewise-linear calibration of the continuous values. Each feature has its own number of quantiles.
        numContinuous := continuous.Shape().Dimensions[1]
        for contIdx := 0; contIdx < numContinuous; contIdx++ {
            // Take one column at a time of the continuous values.
            split := Slice(continuous, AxisRange(), AxisRange(contIdx, contIdx+1))
            featureName := adult.Data.QuantilesFeatures[contIdx]
            calibrationCtx := ctx.In(fmt.Sprintf("continuous_%d_%s", contIdx, featureName))
            quantiles := adult.Data.Quantiles[contIdx]
            layers.AssertQuantilesForPWLCalibrationValid(quantiles)
            calibrated := layers.PieceWiseLinearCalibration(calibrationCtx, split, Const(g, quantiles), *flagTrainableCalibration)
            calibrated.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
            allEmbeddings = append(allEmbeddings, calibrated)
        }
    }
    layer := Concatenate(allEmbeddings, -1)
    layer.AssertDims(batchSize, -1) // 2-dim tensor, with batch size as the leading dimension (-1 means it is not checked).
    
    layer = layers.DenseWithBias(ctx.In(fmt.Sprintf("DenseLayer_%d", 0)), layer, *flagNumNodes)
    for ii := 1; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("DenseLayer_%d", ii))
        // Add layer with residual connection.
        tmp := Sigmoid(layer)
        if *flagDropoutRate > 0 {
            tmp = layers.Dropout(ctx, tmp, Scalar(g, ModelDType, *flagDropoutRate))
        }
        tmp = layers.DenseWithBias(ctx, tmp, *flagNumNodes)
        layer = Add(layer, tmp)  // Residual connections
    }
    layer = Sigmoid(layer)
    logits := layers.DenseWithBias(ctx.In("DenseFinal"), layer, 1)
    logits.AssertDims(batchSize, 1) // 2-dim tensor, with batch size as the leading dimension.
    return []*Node{logits}
}

%%
adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    

// Let's just check that we get the right shape from the model function, wihtout any real data.
graph := manager.NewGraph("test")
ctx := context.NewContext(manager)
ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

// Create placeholder (parameters) graph nodes, just to test the graph building is working.
inputs := []*Node{
    // Categorical: shaped [batch_size, num_categorical]
    graph.Parameter("categorical", shapes.Make(shapes.Int64, *flagBatchSize, len(adult.Data.VocabulariesFeatures))),
    // Continuous: shaped [batch_size, num_continuos]
    graph.Parameter("continuous", shapes.Make(shapes.Float32, *flagBatchSize, len(adult.Data.QuantilesFeatures))),
    // Weights: shaped [batch_size, 1]
    graph.Parameter("weights", shapes.Make(shapes.Float32, *flagBatchSize, 1)),    
}
logits := ModelGraph(ctx, nil, inputs)
fmt.Printf("Logits shape for batch_size=%d: %s\n", *flagBatchSize, logits[0].Shape())

Logits shape for batch_size=128: (Float32)[128 1]


## Training Loop

We can create a training loop with only a `Manager`, a `Context` (for the model varibles) and the `ModelGraph` function.

To make it more interesting we also add the following:

* Accuracy metrics for training and testing.
* Checkpoints -- so trained model can be saved, and reloaded.
* A progress-bar that also shows training metrics.
* We dynamically plot how the loss and accuracy evolve.

First we define the corresponding flags and the `trainModel` function, and run it for very few steps to make sure
it is working.

In [6]:
import (
    "fmt"
    "io"
    "time"

    . "github.com/gomlx/gomlx/graph"

    "github.com/gomlx/gomlx/examples/adult"
    "github.com/gomlx/gomlx/examples/notebook/gonb/margaid"
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
    "github.com/gomlx/gomlx/types/shapes"
    "github.com/gomlx/gomlx/types/slices"
    "github.com/gomlx/gomlx/types/tensor"
    "github.com/janpfeifer/gonb/gonbui"
)

var (
    flagOptimizer      = flag.String("optimizer", "adam", "Type of optimizer to use: 'sgd' or 'adam'")
    flagLearningRate   = flag.Float64("learning_rate", 0.001, "Initial learning rate.")
    flagCheckpoint     = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagPlots          = flag.Bool("plots", true, "Plots during training: perform periodic evaluations, "+
                                   "save results if --checkpoint is set and draw plots, if in a Jupyter notebook.")
)

func trainModel() {
    // Fixes directories.
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    *flagCheckpoint = data.ReplaceTildeInDir(*flagCheckpoint)
    if *flagCheckpoint != "" && !path.IsAbs(*flagCheckpoint) {
        *flagCheckpoint = path.Join(*flagDataDir, *flagCheckpoint)
    }

    // Load data and create datasets.
    adult.LoadAndPreprocessData(*flagDataDir, *flagNumQuantiles, *flagForceDownload, *flagVerbosity)    
    trainDS, trainEvalDS, testEvalDS := BuildDatasets(manager)

    // Context holds the variables and optionally hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).Dir(*flagCheckpoint).Keep(*flagCheckpointKeep).Done()
        if err != nil { panic(err) }
        fmt.Printf("Checkpointing model to %q\n", checkpoint.Dir())
        globalStep := optimizers.GetGlobalStep(ctx)
        if globalStep != 0 {
            fmt.Printf("Restarting training from global_step=%d\n", globalStep)
        }
    }

    // Pick a known optimizer.
    optimizerFn, found := optimizers.KnownOptimizers[*flagOptimizer]
    if !found {
        log.Fatalf("Unknown optimizer %q, please use one of %v",
            *flagOptimizer, slices.Keys(optimizers.KnownOptimizers))
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    trainer := train.NewTrainer(manager, ctx, ModelGraph, losses.BinaryCrossentropyLogits,
        optimizerFn(),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint: checkpoint every 1 minute of training.
    if checkpoint != nil {
        period := time.Minute * 1
        train.PeriodicCallback(loop, period, true, "saving checkpoint", 100,
            func(loop *train.Loop, metrics []tensor.Tensor) error {
                fmt.Printf("\n[saving checkpoint@%d] [median train step (ms): %d]\n", loop.LoopStep, loop.MedianTrainStepDuration().Milliseconds())
                return checkpoint.Save()
            })
    }

	// Attach a margaid plots: plot points at exponential steps.
    // Points (metrics) are saved along the checkpoint directory (if one is given).
	if *flagPlots {
		_ = margaid.NewDefault(loop, checkpoint.Dir(), 100, 1.1, trainEvalDS, testEvalDS)
	}

    // Run the given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    if err != nil { panic(err) }
    fmt.Printf("\t[Step %d] median train step: %d microseconds\n", loop.LoopStep, loop.MedianTrainStepDuration().Microseconds())

    // Print a final evaluation on train and test datasets.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    if err != nil { panic(err) }
    fmt.Println()
}

// Notice command line flags are passed in the %% notebook command. We set --plot=false here to disable plotting
// since this is only a quick test that our train() loop is working. See below the final run for a full training.
%% --steps=500 --plots=false
trainModel()

Training (500 steps):  100% [========================================] (174 steps/s) [loss=0.383] [~loss=0.385] [~acc=82.41%]        
	[Step 500] median train step: 1067 microseconds

Results on batched train:
	Mean Loss (#loss): 0.356
	Mean Accuracy (#acc): 84.03%
Results on test:
	Mean Loss (#loss): 0.353
	Mean Accuracy (#acc): 84.17%



## Final run with 5K steps

With everything working, we can do our final run.

> **Note** here is where someone might want to hyperparameter tune, trying out different hyperparameters.

In [7]:
// Remove previously trained model -- skip this cell, if you want to continue training.
!rm -rf ~/work/uci-adult/base_model

In [8]:
%% --steps=5000 --checkpoint base_model
trainModel()

Checkpointing model to "/home/jupyter/work/uci-adult/base_model"


Training (5000 steps):  100% [========================================] (300 steps/s) [loss=0.197] [~loss=0.272] [~acc=87.40%]        

[saving checkpoint@5000] [median train step (ms): 0]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L1.786294e+02,1.715936e+02 L2.773355e+02,1.858596e+02 L3.506075e+02,1.853263e+02 L4.099543e+02,1.946593e+02 L4.607660e+02,2.129253e+02 L5.055983e+02,2.159917e+02 L5.461769e+02,2.265247e+02 L5.835664e+02,2.322579e+02 L6.184621e+02,2.353911e+02 L6.513380e+02,2.411908e+02 L6.825291e+02,2.438574e+02 L7.123676e+02,2.476571e+02 L7.410809e+02,2.523237e+02 L7.688274e+02,2.524570e+02 L7.957190e+02,2.521904e+02 L8.218897e+02,2.504571e+02 L8.474287e+02,2.508571e+02 L8.663200e+02,2.519904e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.514860e+01 L1.786294e+02,1.735375e+02 L2.773355e+02,1.906030e+02 L3.506075e+02,1.920695e+02 L4.099543e+02,1.991358e+02 L4.607660e+02,2.183343e+02 L5.055983e+02,2.199342e+02 L5.461769e+02,2.306001e+02 L5.835664e+02,2.302002e+02 L6.184621e+02,2.351332e+02 L6.513380e+02,2.407328e+02 L6.825291e+02,2.408661e+02 L7.123676e+02,2.452658e+02 L7.410809e+02,2.464657e+02 L7.688274e+02,2.439325e+02 L7.957190e+02,2.472656e+02 L8.218897e+02,2.387330e+02 L8.474287e+02,2.492655e+02 L8.663200e+02,2.492655e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.496475e+01 L1.786294e+02,4.589438e+01 L2.773355e+02,1.398380e+02 L3.506075e+02,1.756551e+02 L4.099543e+02,1.909674e+02 L4.607660e+02,2.007064e+02 L5.055983e+02,2.137505e+02 L5.461769e+02,2.185449e+02 L5.835664e+02,2.257745e+02 L6.184621e+02,2.278470e+02 L6.513380e+02,2.394653e+02 L6.825291e+02,2.432039e+02 L7.123676e+02,2.450895e+02 L7.410809e+02,2.441317e+02 L7.688274e+02,2.525009e+02 L7.957190e+02,2.548000e+02 L8.218897e+02,2.463987e+02 L8.474287e+02,2.483658e+02 L8.663200e+02,2.543107e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.680453e+02,0 L1.680453e+02,-6 M2.560033e+02,0 L2.560033e+02,-6 M3.184106e+02,0 L3.184106e+02,-6 M3.668174e+02,0 L3.668174e+02,-6 M4.063686e+02,0 L4.063686e+02,-6 M4.398087e+02,0 L4.398087e+02,-6 M4.687758e+02,0 L4.687758e+02,-6 M4.943267e+02,0 L4.943267e+02,-6 M5.171826e+02,0 L5.171826e+02,-6 M5.378584e+02,0 L5.378584e+02,-6 M6.675479e+02,0 L6.675479e+02,-6 M7.555060e+02,0 L7.555060e+02,-6 M8.179132e+02,0 L8.179132e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 Steps 0.780 0.810 0.840 0.870 accuracy accuracy metrics Eval on batched train: Mean Accuracy Eval on test: Mean Accuracy Train: Moving Average Accuracy

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.457363e+02 L1.786294e+02,1.079341e+02 L2.773355e+02,8.277740e+01 L3.506075e+02,8.157413e+01 L4.099543e+02,6.127238e+01 L4.607660e+02,4.986944e+01 L5.055983e+02,4.898977e+01 L5.461769e+02,3.558468e+01 L5.835664e+02,3.031549e+01 L6.184621e+02,2.862244e+01 L6.513380e+02,2.130808e+01 L6.825291e+02,1.975321e+01 L7.123676e+02,1.684195e+01 L7.410809e+02,1.240229e+01 L7.688274e+02,1.089916e+01 L7.957190e+02,9.633696e+00 L8.218897e+02,1.568073e+01 L8.474287e+02,7.913151e+00 L8.663200e+02,7.081782e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.412075e+02 L1.786294e+02,1.033201e+02 L2.773355e+02,7.867322e+01 L3.506075e+02,7.761608e+01 L4.099543e+02,5.954706e+01 L4.607660e+02,4.849424e+01 L5.055983e+02,4.754069e+01 L5.461769e+02,3.519788e+01 L5.835664e+02,3.092012e+01 L6.184621e+02,2.919878e+01 L6.513380e+02,2.296735e+01 L6.825291e+02,2.171394e+01 L7.123676e+02,1.957979e+01 L7.410809e+02,1.543917e+01 L7.688274e+02,1.419414e+01 L7.957190e+02,1.330909e+01 L8.218897e+02,1.852664e+01 L8.474287e+02,1.162763e+01 L8.663200e+02,1.113814e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.786294e+02,1.983609e+02 L2.773355e+02,1.206794e+02 L3.506075e+02,9.243393e+01 L4.099543e+02,7.309505e+01 L4.607660e+02,6.040839e+01 L5.055983e+02,5.020129e+01 L5.461769e+02,4.114269e+01 L5.835664e+02,3.366788e+01 L6.184621e+02,3.355256e+01 L6.513380e+02,2.362708e+01 L6.825291e+02,1.882275e+01 L7.123676e+02,1.487754e+01 L7.410809e+02,1.790326e+01 L7.688274e+02,9.998218e+00 L7.957190e+02,7.845914e+00 L8.218897e+02,1.584750e+01 L8.474287e+02,9.741301e+00 L8.663200e+02,5.200000e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.680453e+02,0 L1.680453e+02,-6 M2.560033e+02,0 L2.560033e+02,-6 M3.184106e+02,0 L3.184106e+02,-6 M3.668174e+02,0 L3.668174e+02,-6 M4.063686e+02,0 L4.063686e+02,-6 M4.398087e+02,0 L4.398087e+02,-6 M4.687758e+02,0 L4.687758e+02,-6 M4.943267e+02,0 L4.943267e+02,-6 M5.171826e+02,0 L5.171826e+02,-6 M5.378584e+02,0 L5.378584e+02,-6 M6.675479e+02,0 L6.675479e+02,-6 M7.555060e+02,0 L7.555060e+02,-6 M8.179132e+02,0 L8.179132e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 Steps 0.300 0.400 0.500 loss loss metrics Eval on batched train: Mean Loss Eval on test: Mean Loss Train: Moving Average Loss

	[Step 5000] median train step: 954 microseconds

Results on batched train:
	Mean Loss (#loss): 0.275
	Mean Accuracy (#acc): 87.29%
Results on test:
	Mean Loss (#loss): 0.279
	Mean Accuracy (#acc): 87.16%



## Extend training another 5K steps

Since the model training went well, and it doesn't seem to be yet terribly overfiting, 
let's train further, another 5k steps, for 10K steps in total.

Notice the plots continue from where it stopped.

Unfortunately, it doesn't help (the accuracy on the test set doesn't improve), 5k steps was already enough.


In [9]:
%% --steps=5000 --checkpoint base_model
trainModel()

Checkpointing model to "/home/jupyter/work/uci-adult/base_model"
Restarting training from global_step=5000


Training (5000 steps):  100% [========================================] (369 steps/s) [loss=0.297] [~loss=0.273] [~acc=87.14%]        

[saving checkpoint@10000] [median train step (ms): 0]


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L1.544041e+02,1.669138e+02 L2.382534e+02,1.807786e+02 L3.004969e+02,1.802603e+02 L3.509112e+02,1.893308e+02 L3.940749e+02,2.070831e+02 L4.321593e+02,2.100633e+02 L4.666302e+02,2.203001e+02 L4.983920e+02,2.258719e+02 L5.280354e+02,2.289170e+02 L5.559630e+02,2.345537e+02 L5.824593e+02,2.371452e+02 L6.078067e+02,2.408381e+02 L6.321982e+02,2.453734e+02 L6.557685e+02,2.455030e+02 L6.786125e+02,2.452439e+02 L7.008441e+02,2.435593e+02 L7.225391e+02,2.439481e+02 L7.385870e+02,2.450495e+02 L7.437834e+02,2.454383e+02 L7.646305e+02,2.448551e+02 L7.851406e+02,2.440776e+02 L8.053477e+02,2.490664e+02 L8.252913e+02,2.470579e+02 L8.449900e+02,2.480298e+02 L8.644679e+02,2.455030e+02 L8.663200e+02,2.472523e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.486880e+01 L1.544041e+02,1.688031e+02 L2.382534e+02,1.853886e+02 L3.004969e+02,1.868139e+02 L3.509112e+02,1.936814e+02 L3.940749e+02,2.123400e+02 L4.321593e+02,2.138949e+02 L4.666302e+02,2.242608e+02 L4.983920e+02,2.238721e+02 L5.280354e+02,2.286664e+02 L5.559630e+02,2.341085e+02 L5.824593e+02,2.342381e+02 L6.078067e+02,2.385140e+02 L6.321982e+02,2.396802e+02 L6.557685e+02,2.372183e+02 L6.786125e+02,2.404576e+02 L7.008441e+02,2.321649e+02 L7.225391e+02,2.424012e+02 L7.385870e+02,2.424012e+02 L7.437834e+02,2.434378e+02 L7.646305e+02,2.342381e+02 L7.851406e+02,2.421421e+02 L8.053477e+02,2.427900e+02 L8.252913e+02,2.431787e+02 L8.449900e+02,2.430492e+02 L8.644679e+02,2.427900e+02 L8.663200e+02,2.438266e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.469012e+01 L1.544041e+02,4.474986e+01 L2.382534e+02,1.360513e+02 L3.004969e+02,1.708611e+02 L3.509112e+02,1.857428e+02 L3.940749e+02,1.952079e+02 L4.321593e+02,2.078851e+02 L4.666302e+02,2.125447e+02 L4.983920e+02,2.195709e+02 L5.280354e+02,2.215851e+02 L5.559630e+02,2.328766e+02 L5.824593e+02,2.365101e+02 L6.078067e+02,2.383426e+02 L6.321982e+02,2.374118e+02 L6.557685e+02,2.455456e+02 L6.786125e+02,2.477801e+02 L7.008441e+02,2.396150e+02 L7.225391e+02,2.415268e+02 L7.385870e+02,2.473045e+02 L7.437834e+02,2.460218e+02 L7.646305e+02,2.438632e+02 L7.851406e+02,2.465966e+02 L8.053477e+02,2.458586e+02 L8.252913e+02,2.505763e+02 L8.449900e+02,2.449924e+02 L8.644679e+02,2.548000e+02 L8.663200e+02,2.420079e+02 "/> <path d="M1.768000e+01,0 L1.768000e+01,-6 M1.454130e+02,0 L1.454130e+02,-6 M2.201321e+02,0 L2.201321e+02,-6 M2.731461e+02,0 L2.731461e+02,-6 M3.142670e+02,0 L3.142670e+02,-6 M3.478651e+02,0 L3.478651e+02,-6 M3.762720e+02,0 L3.762720e+02,-6 M4.008791e+02,0 L4.008791e+02,-6 M4.225842e+02,0 L4.225842e+02,-6 M442,0 L442,-6 M4.595637e+02,0 L4.595637e+02,-6 M5.697330e+02,0 L5.697330e+02,-6 M6.444521e+02,0 L6.444521e+02,-6 M6.974661e+02,0 L6.974661e+02,-6 M7.385870e+02,0 L7.385870e+02,-6 M7.721851e+02,0 L7.721851e+02,-6 M8.005920e+02,0 L8.005920e+02,-6 M8.251991e+02,0 L8.251991e+02,-6 M8.469042e+02,0 L8.469042e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 " vector-effect="non-scaling-stroke"/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 Steps 0.780 0.810 0.840 0.870 accuracy accuracy metrics Eval on batched train: Mean Accuracy Eval on test: Mean Accuracy Train: Moving Average Accuracy

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.459448e+02 L1.544041e+02,1.113122e+02 L2.382534e+02,8.673420e+01 L3.004969e+02,8.555860e+01 L3.509112e+02,6.572383e+01 L3.940749e+02,5.458317e+01 L4.321593e+02,5.372373e+01 L4.666302e+02,4.062698e+01 L4.983920e+02,3.547899e+01 L5.280354e+02,3.382489e+01 L5.559630e+02,2.667877e+01 L5.824593e+02,2.515966e+01 L6.078067e+02,2.231536e+01 L6.321982e+02,1.797783e+01 L6.557685e+02,1.650927e+01 L6.786125e+02,1.527291e+01 L7.008441e+02,2.118085e+01 L7.225391e+02,1.359194e+01 L7.385870e+02,1.277970e+01 L7.437834e+02,1.250429e+01 L7.646305e+02,1.508003e+01 L7.851406e+02,1.343675e+01 L8.053477e+02,1.133662e+01 L8.252913e+02,1.042012e+01 L8.449900e+02,9.246373e+00 L8.644679e+02,9.597070e+00 L8.663200e+02,9.109989e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.415201e+02 L1.544041e+02,1.068044e+02 L2.382534e+02,8.272441e+01 L3.004969e+02,8.169159e+01 L3.509112e+02,6.403819e+01 L3.940749e+02,5.323960e+01 L4.321593e+02,5.230799e+01 L4.666302e+02,4.024908e+01 L4.983920e+02,3.606971e+01 L5.280354e+02,3.438797e+01 L5.559630e+02,2.829987e+01 L5.824593e+02,2.707529e+01 L6.078067e+02,2.499023e+01 L6.321982e+02,2.094486e+01 L6.557685e+02,1.972846e+01 L6.786125e+02,1.886377e+01 L7.008441e+02,2.396131e+01 L7.225391e+02,1.722098e+01 L7.385870e+02,1.674275e+01 L7.437834e+02,1.639978e+01 L7.646305e+02,1.979623e+01 L7.851406e+02,1.814793e+01 L8.053477e+02,1.620771e+01 L8.252913e+02,1.638790e+01 L8.449900e+02,1.538761e+01 L8.644679e+02,1.630722e+01 L8.663200e+02,1.583284e+01 "/> <path d="M1.768000e+01,2.548000e+02 L1.544041e+02,1.996591e+02 L2.382534e+02,1.237644e+02 L3.004969e+02,9.616861e+01 L3.509112e+02,7.727456e+01 L3.940749e+02,6.487971e+01 L4.321593e+02,5.490739e+01 L4.666302e+02,4.605715e+01 L4.983920e+02,3.875428e+01 L5.280354e+02,3.864160e+01 L5.559630e+02,2.894443e+01 L5.824593e+02,2.425060e+01 L6.078067e+02,2.039614e+01 L6.321982e+02,2.335226e+01 L6.557685e+02,1.562905e+01 L6.786125e+02,1.352625e+01 L7.008441e+02,2.134379e+01 L7.225391e+02,1.537804e+01 L7.385870e+02,1.094120e+01 L7.437834e+02,1.138961e+01 L7.646305e+02,1.384303e+01 L7.851406e+02,1.511410e+01 L8.053477e+02,1.441094e+01 L8.252913e+02,1.028394e+01 L8.449900e+02,1.120085e+01 L8.644679e+02,5.200000e+00 L8.663200e+02,1.150323e+01 " vector-effect="non-scaling-stroke"/> <path d="M1.768000e+01,0 L1.768000e+01,-6 M1.454130e+02,0 L1.454130e+02,-6 M2.201321e+02,0 L2.201321e+02,-6 M2.731461e+02,0 L2.731461e+02,-6 M3.142670e+02,0 L3.142670e+02,-6 M3.478651e+02,0 L3.478651e+02,-6 M3.762720e+02,0 L3.762720e+02,-6 M4.008791e+02,0 L4.008791e+02,-6 M4.225842e+02,0 L4.225842e+02,-6 M442,0 L442,-6 M4.595637e+02,0 L4.595637e+02,-6 M5.697330e+02,0 L5.697330e+02,-6 M6.444521e+02,0 L6.444521e+02,-6 M6.974661e+02,0 L6.974661e+02,-6 M7.385870e+02,0 L7.385870e+02,-6 M7.721851e+02,0 L7.721851e+02,-6 M8.005920e+02,0 L8.005920e+02,-6 M8.251991e+02,0 L8.251991e+02,-6 M8.469042e+02,0 L8.469042e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 " vector-effect="non-scaling-stroke"/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 Steps 0.300 0.400 0.500 loss loss metrics Eval on batched train: Mean Loss Eval on test: Mean Loss Train: Moving Average Loss

	[Step 10000] median train step: 948 microseconds

Results on batched train:
	Mean Loss (#loss): 0.270
	Mean Accuracy (#acc): 87.39%
Results on test:
	Mean Loss (#loss): 0.278
	Mean Accuracy (#acc): 87.23%

